In [2]:
import torch
import torch.nn as nn
import torch.optim as optim



In [10]:
data_test=  []
lab_test = []
ones = True
for i in range(10):
    tdat  = []
    tlab = []
    
    for i in range(90):
        tdat.append(i)
        if ones:
            tlab.append(1)
        else:
            tlab.append(0)
    ones = not ones
    data_test.append(tdat)
    lab_test.append(tlab)
X_train = torch.FloatTensor(data_test)
X_test = X_train
y_train = torch.FloatTensor(lab_test)
y_test = y_train
import numpy as np

# Create an array of length 10, with each element being an array of length 90
data = np.random.rand(10, 90)

# Create labels for each element (assume binary labels)
labels = np.random.randint(0, 1, size=10)

# Split the data into X_train and y_train
X_train = data
y_train = labels
print(X_train)
print(y_train)
# Convert X_train and y_train to PyTorch tensors (if using PyTorch)
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float().unsqueeze(1)
X_train = X_train_tensor
y_train = y_train_tensor


[[6.22936897e-01 3.78589048e-01 9.47999471e-02 1.55455681e-01
  5.97628519e-01 3.58894322e-01 2.76084583e-01 8.39724523e-02
  4.93775806e-02 4.84753526e-01 1.12686422e-01 3.49192719e-01
  7.21388186e-01 1.23681515e-01 4.65296363e-02 4.64568596e-01
  1.84492600e-01 7.76585326e-01 4.22661906e-01 7.83815686e-01
  8.44135123e-01 8.76050656e-01 4.78875013e-01 7.56776452e-01
  6.56955454e-01 9.36665324e-01 2.68762339e-01 8.32514986e-01
  8.00474616e-01 4.49355204e-01 9.57458181e-01 2.67938239e-01
  4.49342627e-01 1.12386945e-01 5.13748838e-01 8.37681662e-01
  4.64795306e-01 3.83343978e-01 9.87581761e-04 6.42646884e-01
  6.18517837e-01 6.66680509e-01 8.16617939e-01 1.37591535e-01
  2.10125357e-01 3.74948590e-01 1.96491166e-02 4.48822319e-01
  3.20541615e-01 4.75648296e-01 5.28422585e-01 7.38011703e-01
  7.11739402e-02 3.88158100e-01 8.45141295e-01 7.74519341e-01
  1.95103587e-01 4.85449403e-01 1.31832925e-01 7.52561506e-02
  3.65426391e-01 4.57822498e-01 6.40268990e-02 1.07331876e-01
  9.2600

In [11]:
# Define the model architecture
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(90, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Create an instance of the model
model = BinaryClassifier()

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
num_epochs = 10
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
with torch.no_grad():
    outputs = model(X_test)
    predicted = (outputs >= 0.5).float()
    accuracy = (predicted == y_test).float().mean()
    print(f'Test Accuracy: {accuracy:.4f}')

# Make predictions
with torch.no_grad():
    outputs = model(X_new)
    predicted = (outputs >= 0.5).float()

Epoch [1/10], Loss: 0.7034
Epoch [2/10], Loss: 0.6764
Epoch [3/10], Loss: 0.6524
Epoch [4/10], Loss: 0.6302
Epoch [5/10], Loss: 0.6097
Epoch [6/10], Loss: 0.5902
Epoch [7/10], Loss: 0.5727
Epoch [8/10], Loss: 0.5554
Epoch [9/10], Loss: 0.5378
Epoch [10/10], Loss: 0.5191
Test Accuracy: 0.5000


NameError: name 'X_new' is not defined